# Decomposition Methods: Metro Transit Approach Applied to Wisconsin TSP

Produced by Wylie Timmerman with the WMATA Datamart Bus Priority Team
Draft: July 16, 2021

## Introduction

In this notebook, we'll see the application of the Metro Transit decomposition approach to Wisconsin Avenue bus routes, along with some light comparisons of pre- and post-TSP decomposition values.

## Environment Setup



In [ ]:
# Libraries
import os, sys, glob, pandas as pd, geopandas as gpd
from dotenv import dotenv_values
import pyarrow as pa
import pyarrow.parquet as pq
import datetime as dt
from itertools import product
import numpy as np
from plotly.offline import plot
import plotly.graph_objs as go
import plotly.express as px
import plotly as py
py.offline.init_notebook_mode()
py.io.renderers.default='notebook'
import statsmodels.formula.api as smf

In [ ]:
# Paths
os.environ["GDAL_DATA"] = os.environ["CONDA_PREFIX"] + "\Library\share\gdal"
path_working = r"C:\Users\WylieTimmerman\Documents\projects_local\WMATA_AVL_datamart"
os.chdir(os.path.join(path_working))
sys.path.append(r"C:\Users\WylieTimmerman\Documents\projects_local\WMATA_AVL_datamart")
path_sp = r"C:\OD\Foursquare ITP\Projects - WMATA Datamart\Task 3 - Bus Priority"
path_source_data = os.path.join(path_sp,"data","00-Raw")
path_processed_data = os.path.join(path_sp, "data","02-Processed")
# Server credentials
config = dotenv_values(os.path.join(path_working, '.env'))

# Globals
wmata_crs = 2248

# Load wmatarawnav library
import wmatarawnav as wr

We'll load previously decomposed trips on Wisconsin Avenue routes 30N, 30S, 33, and 31 from February through May 2021. Each record in the loaded table is uniquely defined by :

* Trip Instance (identified by *filename* and *index_run_start*)
* Trip Segment (identified by *trip_seg*: either the stop window area +/-150 ft from stop, or the segment between these windows)
* Travel Time Decomposition (identified by *full_decomp*: this column differentiates between freeflow and delay in non-passenger time)

In [ ]:
rawnav_run_decomp = (
    pq.read_table(
        source=os.path.join(path_sp,"data","01-Interim","wisconsin_decomp_mt.parquet"),
        use_pandas_metadata = True
    )
    .to_pandas()
    # As a bit of proofing, we confirm this is int32
    .reset_index()
)



We'll filter to data between February 1st and April 30th.

In [ ]:
# We parsed through the end of may, but want a month and a half on each end only.
rawnav_run_decomp = (
    rawnav_run_decomp
    .loc[
        (rawnav_run_decomp.start_date_time.dt.tz_localize(None) < np.datetime64('2021-04-30')) &
        (rawnav_run_decomp.start_date_time.dt.tz_localize(None) >= np.datetime64('2021-02-01')) # the february files include some january 31 dates
    ]
)
    

Below we show the first 10 records belonging to one trip instance in particular. Note that in some cases delay values may be negative. This is because we've calculated freeflow as the 95th percentile of travel time through a stop; as a result, some trip instances will cover the area faster than freeflow speed.

In [ ]:
rawnav_run_decomp.head(10)

## Data Prep for Analysis

Next, we'll add identifiers to trips that will be useful for our analysis; in particular, whether a trip is in general eligible to receive TSP based on its route, direction, time period, and service day. We'll use March 15, 2021 as the breakpoint for when TSP is activated or not. We'll perform data quality checks at the end of this process.

The table below shows categories of trips that may receive TSP; even though Metrobus Route 31 won't receive TSP, we want to pick analogous directions and times for the comparison. We'll also define an 'off-direction' for the TSP peak-direction/peak-time so that we can compare changes in these cases against the direction and time eligible for TSP.

In [ ]:
# TODO: add additional identifiers somehow for if the vehicle is on wisconsin
# TODO: Update to reflect that periods defined not on trip start time, but time when vehicle reached a TSP intersection.

tsp_conditions = wr.tribble(
        ['route','direction_wmata_schedule', 'service_day','time_period','tsp_dir_time', 'tsp_offdir_time'],
        "30N",                       "EAST",     "Weekday",    "AM Peak",           True,            False,
        "30N",                       "WEST",     "Weekday",    "PM Peak",           True,            False,
        "30S",                       "EAST",     "Weekday",    "AM Peak",           True,            False,
        "30S",                       "WEST",     "Weekday",    "PM Peak",           True,            False,
         "33",                      "SOUTH",     "Weekday",    "AM Peak",           True,            False,
         "33",                      "NORTH",     "Weekday",    "PM Peak",           True,            False,
         "31",                      "SOUTH",     "Weekday",    "AM Peak",           True,            False,
         "31",                      "NORTH",     "Weekday",    "PM Peak",           True,            False,
        "30N",                       "WEST",     "Weekday",    "AM Peak",           False,            True,
        "30N",                       "EAST",     "Weekday",    "PM Peak",           False,            True,
        "30S",                       "WEST",     "Weekday",    "AM Peak",           False,            True,
        "30S",                       "EAST",     "Weekday",    "PM Peak",           False,            True,
         "33",                      "NORTH",     "Weekday",    "AM Peak",           False,            True,
         "33",                      "SOUTH",     "Weekday",    "PM Peak",           False,            True,
         "31",                      "NORTH",     "Weekday",    "AM Peak",           False,            True,
         "31",                      "SOUTH",     "Weekday",    "PM Peak",           False,            True
    )

tsp_conditions

The steps below are those used to add these TSP identifiers and others to the data.

We'll also defined a set of holidays during the analysis period to convert to sunday or saturdays, respectively. We won't distinguish saturday and saturday supplemental service here.

In [ ]:
rawnav_run_decomp_2 = (
    rawnav_run_decomp
    .assign(
        is_tsp_route = lambda x: x.route.isin(['30N','30S','33']),
        tsp_period = lambda x: 
            np.where(
                x.start_date_time.dt.tz_localize(None) < np.datetime64('2021-03-15'),
                "pre_tsp",
                "post_tsp"
            ),
        dow = lambda x: x.start_date_time.dt.dayofweek,
        service_day = lambda x: 
            np.select(
                [
                    x.dow.isin([0,1,2,3,4]),
                    x.dow.eq(5),
                    x.dow.eq(6)
                ],
                [
                    'Weekday',
                    'Saturday',
                    'Sunday'
                ]
            )
    )
    .assign(
        service_day = lambda x:
            # we'll set President's day to saturday 
            np.where(
                x.start_date_time.dt.tz_localize(None) == np.datetime64('2021-02-15'),
                'Saturday',
                x.service_day
            ),
        trip_hour = lambda x: x.start_date_time.dt.hour,
        # TODO: double check timestamps/timezone on conversion back from 
        # numpy
        time_period = lambda x: 
            pd.cut(
                x.trip_hour,
                bins = pd.IntervalIndex.from_tuples(
                    [
                        (0,4),
                        (4,6),
                        (6,9),
                        (9,15),
                        (15,19),
                        (19,23),
                        (23,999)
                    ]
                ),
                include_lowest=True, 
                retbins = False
            )
            .astype("category")
            .cat.rename_categories(
                ['Late Night1',
                'Early AM',
                'AM Peak',
                'Midday',
                'PM Peak',
                'Evening',
                'Late Night2']
            )
    )
    .merge(
        tsp_conditions,
        on = ['route','direction_wmata_schedule', 'service_day','time_period'],
        how = "left"
    )
    .assign(
        tsp_dir_time = lambda x : x.tsp_dir_time.fillna(False),
        overall_dir = lambda x : 
            np.where(
                x.direction_wmata_schedule.isin(['EAST','SOUTH']),
                "Southbound",
                "Northbound"
            )
    )
)

A sample of the decomposition values with these additional identifers are shown below.

In [ ]:
rawnav_run_decomp_2.head(100)

Next, we'll also filter to the trips and stops that are on the Wisconsin corridor and have data for the complete set of stops on this corridor; at a later date, we will evalutate entire routes. Because of what we've found with GPS errors around the terminal, we'll drop the terminal at Friendship Heights as a corridor stop after we import the list of corridor stops.

In [ ]:
wisconsin_corr_stops = (
    pd.read_csv(os.path.join(path_processed_data,"wisconsin_corridor_stops.csv"))
    .query('stop_id != 32089')
)

In [ ]:
wisconsin_corr_stops.head()

In [ ]:
wisconsin_corr_grp = wisconsin_corr_stops.groupby(['route','direction'])
rawnav_run_decomp_2_grp = rawnav_run_decomp_2.groupby(['route','direction_wmata_schedule','filename','index_run_start'])

rawnav_run_decomp_2_wisc = pd.DataFrame()

trips_missing_stops = pd.DataFrame()

for name, rawnav_group in rawnav_run_decomp_2_grp:
    # try:
    wisconsin_corr_rtdir_stops = (
        wisconsin_corr_grp
        .get_group((name[0], name[1]))
        .stop_id
        .astype(str)
    )
    
    rawnav_group_stops = (
        rawnav_group
        # filter to just the stops
        .query('(basic_decomp == "Non-Passenger") | (basic_decomp == "Passenger")')
        .trip_seg
    )

    miss_stops = list(set(wisconsin_corr_rtdir_stops) - set(rawnav_group_stops))

    if (len(miss_stops) == 0):
        # if you have all the corridor stops, then we'll filter to those stops and keep
        # that decomposition. Later we may want to separate this part out
        # this regex is a little bespoke, but so i think we'll need to use a different approach for parsing stops and stop segs in the future

        stop_regex = r"(?:^|_)[^\d]*(?:" + "|".join(wisconsin_corr_rtdir_stops) + ")"

        rawnav_group_out = (
            rawnav_group
            .loc[rawnav_group.trip_seg.str.contains(stop_regex, na = False)]
        )

        rawnav_run_decomp_2_wisc = pd.concat([rawnav_run_decomp_2_wisc, rawnav_group_out])
    else:
        trip_miss_deets = pd.DataFrame({"filename" : name[2], "index_run_start" : name[3], 'route' : name[0],'direction_wmata_schedule' : name[1], "miss_stop" : miss_stops})
        trips_missing_stops = pd.concat([trips_missing_stops,trip_miss_deets])
    

Where did we lose trips? What were the stops that didn't join? if it was mostly 32089, then it's mostly join issue at the terminal and we shouldn't be too worried. But maybe we should filter that out first, not sure...

In [ ]:
count_missing_stops = (
    trips_missing_stops
    .groupby(['miss_stop'])
    .agg(count = ('miss_stop','count'))
)

count_missing_stops

Next, we'll modify this a bit further to remove the portions of routes that are arriving or departing from the bus terminal at Western Ave. Until further processing steps to cleanly identify the start and end of a trip are developed, it is better to exclude these portions of the route from the analysis. Even though we excluded the terminal from our corridor stops earlier, this next step will also remove the segments leading to and from the terminal.

In [ ]:
rawnav_run_decomp_2_fil = (
    rawnav_run_decomp_2_wisc
    #stop id for friendship heights bay used by these buses; this will remove segments leading to and from as well
    .loc[~rawnav_run_decomp_2_wisc.trip_seg.str.contains('32089', na = False)] 
)

We'll aggregate to the trip-level to do some reasonableness filtering on trip instances, keeping all trips with total corridor runtimes between the 1st and 99th percentile.

In [ ]:
trip_times = (
    rawnav_run_decomp_2_fil
    .groupby(['filename','index_run_start'])
    .agg(
        tot_secs = ('secs_tot','sum')
    )
)

trip_times_dist = (
    trip_times
    .agg(
        tot_secs_min = ('tot_secs','min'),
        tot_secs_p01 = ('tot_secs', lambda x: x.quantile(.01)),
        tot_secs_p05 = ('tot_secs', lambda x: x.quantile(.05)),
        tot_secs_p50 = ('tot_secs', lambda x: x.quantile(.50)),
        tot_secs_p95 = ('tot_secs', lambda x: x.quantile(.95)),
        tot_secs_p99 = ('tot_secs', lambda x: x.quantile(.99)),
        tot_secs_max = ('tot_secs', 'max'),
    )
)

trip_times_dist

In [ ]:
trip_times_low = (
    trip_times_dist
    .loc['tot_secs_p01','tot_secs']
)

trip_times_high = (
    trip_times_dist
    .loc['tot_secs_p99','tot_secs']
)

trip_times_fil = (
    trip_times
    .loc[(trip_times.tot_secs >= trip_times_low) & (trip_times.tot_secs <= trip_times_high)]
    .reset_index()
)

rawnav_run_decomp_2_fil = (
    rawnav_run_decomp_2_fil
    .merge(
        trip_times_fil,
        how = 'right',
        on = ['filename','index_run_start']
    )
    .reindex(rawnav_run_decomp_2_fil.columns, axis = "columns")
)

In [ ]:
removed = trip_times.shape[0] - trip_times_fil.shape[0]
print('by filtering to trips between the first and 99th percentile in runtimes, ' + str(removed) + ' trips were removed.')

In [ ]:
rawnav_run_decomp_trip = (
    rawnav_run_decomp_2_fil 
    .groupby(
        ['filename',
        'index_run_start',
        'full_decomp']
    )
    .agg(
        tot_secs = ('secs_tot','sum')
    )
    .reset_index()
)

trip_details = (
    rawnav_run_decomp_2_fil
    .filter(
        [
            'filename',
            'index_run_start',
            'route',
            'direction_wmata_schedule',
            'pattern_destination_wmata_schedule',
            'overall_dir',
            'is_tsp_route',
            'time_period',
            'service_day',
            'start_date_time',
            'tsp_period',
            'tsp_dir_time'
        ],
        axis =  "columns"
    )
    .drop_duplicates()
)

rawnav_run_decomp_trip_2 = (
    rawnav_run_decomp_trip
    .merge(
        trip_details,
        on = ['filename','index_run_start'],
        how = 'left'
    )
)

rawnav_run_decomp_trip_2.head()



In [ ]:
rawnav_run_decomp_trip_2.to_csv("rawnav_run_decomp_trip_2.csv")

## Data Quality Checks

### Counts of Trips
Before we begin analysis, we'll perform a few additional filtering steps to ensure we're reviewing trips without outlier values for speed, distance traveled, or travel time.

How many total trips do we capture in the rawnav data?

In [ ]:
check_val = (
    rawnav_run_decomp
    .filter(items = ['filename','index_run_start'])
    .drop_duplicates()
    .shape[0]
)

print("Number of 30N/30S/33/31 trips in parsed rawnav data: " + str(check_val))

What about trips with data on the Wisconsin corridor?

In [ ]:
check_val = (
    rawnav_run_decomp_trip_2
    .filter(items = ['filename','index_run_start'])
    .drop_duplicates()
    .shape[0]
)

print("Number of 30N/30S/33/31 trips with data on the corridor: " + str(check_val))

### Runtime Comparisons to Trace

What is the runtime we see on the corridor? Does it approximately match what we see in Scott's data?

In [ ]:
last_study = wr.tribble(
    [   'routes','direction',      'time',	'median',	'n95th'],
            '31',	    'NB',	'PM Peak',	    12.8,	   17.3,
    '30N/30S/33',	    'NB',	'PM Peak',	    13.6,	   17.7,
            '31',	    'SB',	'AM Peak',	    11.1,	   14.9,
    '30N/30S/33',	    'SB',	'AM Peak',	    10.8,	   14.4
)

last_study

In [ ]:
trip_times_posttsp = (
    rawnav_run_decomp_trip_2
    .loc[
        (rawnav_run_decomp_trip_2.tsp_dir_time == True) & 
        (rawnav_run_decomp_trip_2.tsp_period == "post_tsp") & 
        (rawnav_run_decomp_trip_2.start_date_time.dt.tz_localize(None) <= np.datetime64('2021-04-15')) # to match previous analysis
    ]
    .assign(
        gen_dir = lambda x: 
            np.where(
                x.direction_wmata_schedule.isin(['EAST','SOUTH']),
                "SB",
                "NB"
            )
    )
    .groupby(['filename','index_run_start','is_tsp_route','overall_dir','time_period'])
    .agg(
        tot_secs = ('tot_secs','sum')
    )
    .reset_index()
    .assign(
        tot_mins = lambda x: x.tot_secs / 60
    )
    .groupby(['is_tsp_route','overall_dir','time_period'])
    .agg(
        tot_mins_p50 = ('tot_mins', lambda x: x.quantile(.50)),
        tot_mins_p95 = ('tot_mins', lambda x: x.quantile(.95))
    )
    .reset_index()
    .assign(
        route_names = lambda x: 
            np.where(
                x.is_tsp_route,
                "30N/30S/33",
                "31"
            )
    )
    .drop(columns = 'is_tsp_route')
)
trip_times_posttsp

These values seem a little high relative to the last study. What if we remove passenger activity to get at runtime?

In [ ]:
# for convenience, we'll just copy the code chunk but add the filter earlier
trip_times_posttsp_nopax = (
    rawnav_run_decomp_trip_2
    .loc[
        (rawnav_run_decomp_trip_2.tsp_dir_time == True) & 
        (rawnav_run_decomp_trip_2.tsp_period == "post_tsp") & 
        (rawnav_run_decomp_trip_2.start_date_time.dt.tz_localize(None) <= np.datetime64('2021-04-15')) &
        (rawnav_run_decomp_trip_2.full_decomp != "Passenger")
    ]
    .groupby(['filename','index_run_start','is_tsp_route','overall_dir','time_period'])
    .agg(
        tot_secs = ('tot_secs','sum')
    )
    .reset_index()
    .assign(
        tot_mins = lambda x: x.tot_secs / 60
    )
    .groupby(['is_tsp_route','overall_dir','time_period'])
    .agg(
        tot_mins_p50 = ('tot_mins', lambda x: x.quantile(.50)),
        tot_mins_p95 = ('tot_mins', lambda x: x.quantile(.95))
    )
    .reset_index()
    .assign(
        route_names = lambda x: 
            np.where(
                x.is_tsp_route,
                "30N/30S/33",
                "31"
            )
    )
    .drop(columns = 'is_tsp_route')
)
trip_times_posttsp_nopax

These values without dwell time/passenger time are still a little higher than the previous study, but not significantly. The remaining difference might be attributed to how the corridor is defined.

### Runtime Trends
Do we see parallel trends in median runtimes week by week? 

In [ ]:
trip_times_trends = (
    rawnav_run_decomp_trip_2
    .loc[(rawnav_run_decomp_trip_2.tsp_dir_time == True) & (rawnav_run_decomp_trip_2.full_decomp != "Passenger")]
    .assign(
        start_date = lambda x: x.start_date_time.dt.round(freq = "24H"),
        weekgroup = lambda x: x.start_date - pd.to_timedelta(x.start_date.dt.dayofweek, unit = 'd')
    )
    .groupby(['filename','index_run_start','is_tsp_route','overall_dir','time_period','tsp_period','weekgroup'])
    .agg(
        tot_secs = ('tot_secs','sum')
    )
    .reset_index()
    .assign(
        tot_mins = lambda x: x.tot_secs / 60
    )
    .groupby(['is_tsp_route','tsp_period','weekgroup'])
    .agg(
        tot_mins_avg = ('tot_mins','mean'),
        tot_mins_p50 = ('tot_mins', lambda x: x.quantile(.50)),
        tot_mins_p95 = ('tot_mins', lambda x: x.quantile(.95))
    )
    .reset_index()
    .assign(
        route_names = lambda x: 
            np.where(
                x.is_tsp_route,
                "30N/30S/33",
                "31"
            )
    )
    .sort_values(['is_tsp_route','weekgroup'])
)

trip_times_trends

In [ ]:
fig0 = px.line(
    trip_times_trends,
    x = "weekgroup",
    y = "tot_mins_avg",
    color = "route_names",
    labels={ # replaces default labels by column name
        "route_names": "Routes",  
        "tot_mins_avg" : "Total Time (Average)",
        "weekgroup" : "Week"
    },
    template = "simple_white"
)

fig0.update_layout(
    font=dict(
        size=18
    )
)

py.offline.iplot(fig0, filename='Overtime')

## Descriptive Stats


### Trip Level
How have median values for the decomposition at the trip level changed by route, direction, and period?

In [ ]:
rawnav_run_decomp_agg = (
    rawnav_run_decomp_trip_2 
    # let's first summarize to trip level 
    .loc[rawnav_run_decomp_trip_2.tsp_dir_time == True]
    .drop(['tsp_dir_time'], axis = 'columns')
    .groupby(['route','direction_wmata_schedule','tsp_period','full_decomp'])
    .agg({
        'tot_secs' : [lambda x: x.quantile(.50)],
        'filename' : ['count']
    })
    .pipe(wr.reset_col_names)
    .query("full_decomp != 'End of Trip Pings'")
    .rename(
        columns = {
            "tot_secs_<lambda>" : "tot_secs_med",
            "filename_count" : "n_count"
        }
    )
)

rawnav_run_decomp_agg['tot_secs_pct'] = (
    rawnav_run_decomp_agg
    .groupby(['route','direction_wmata_schedule','tsp_period'])['tot_secs_med']
    .transform(lambda x: x / x.sum())
)

rawnav_run_decomp_agg_pivot = (
    rawnav_run_decomp_agg
    .pivot(
            index = ['route','direction_wmata_schedule','full_decomp'],
            columns = 'tsp_period',
            values = ['n_count','tot_secs_med','tot_secs_pct']
    )
    .pipe(wr.reset_col_names)
    .assign(
        tot_secs_med_pre_tsp = lambda x: round(x.tot_secs_med_pre_tsp),
        tot_secs_med_post_tsp = lambda x: round(x.tot_secs_med_post_tsp),
        tot_secs_med_diff = lambda x: round(x.tot_secs_med_post_tsp - x.tot_secs_med_pre_tsp),
        tot_secs_pp_diff = lambda x: round((x.tot_secs_pct_post_tsp - x.tot_secs_pct_pre_tsp) * 100).astype(str) + "pp",  
        tot_secs_pct_pre_tsp = lambda x: (round(x.tot_secs_pct_pre_tsp * 100)).astype(str) + "%",
        tot_secs_pct_post_tsp = lambda x: round(x.tot_secs_pct_post_tsp * 100).astype(str) + "%",
    )
    .reindex(
        columns = [
            "route",
            "direction_wmata_schedule",
            "full_decomp",
            "n_count_pre_tsp",
            "n_count_post_tsp",
            "tot_secs_med_pre_tsp",
            "tot_secs_med_post_tsp", 
            "tot_secs_med_diff",
            "tot_secs_pct_pre_tsp",
            "tot_secs_pct_post_tsp",
            "tot_secs_pp_diff"
        ]
    )
    .rename(
        columns = {
            "tot_secs_med_post_tsp" : "Median Total Seconds (Post TSP)",
            "tot_secs_med_pre_tsp" : "Median Total Seconds (Pre TSP)",
            "tot_secs_med_diff" : "Change in Median Total Seconds (Pre to Post)",
            "tot_secs_pct_pre_tsp": "Percentage of Total Time (Pre TSP)",
            "tot_secs_pct_post_tsp" : "Percentage of Total Time (Post TSP)",
            "tot_secs_pp_diff" : "Percentage Point Change in Total Time (Pre to Post)"
        }
    )
)

rawnav_run_decomp_agg.to_csv(r"rawnav_run_decomp_agg.csv")

rawnav_run_decomp_agg_pivot

In [ ]:
def stacked_bar(
    fig_data
):
    fig = go.Figure()

    fig.update_layout(
        template="simple_white",
        xaxis=dict(title_text="Route & Direction"),
        yaxis=dict(title_text="Total Seconds (Median Values)"),
        barmode="stack",
        font=dict(
            size=18
        )
    )

    fig_data = (
        fig_data
        .assign(
            color = lambda x: 
                np.select(
                    [
                        x.full_decomp == "<5 mph",
                        x.full_decomp == ">= 5mph",
                        x.full_decomp == "Delay",
                        x.full_decomp == "Freeflow",
                        x.full_decomp == "Passenger"
                    ],
                    [
                      "#440154",  # <5mph
                      "#21908C", # >=5mph
                      "#9C179E", # delay
                      "#FDE725", # freeflow
                      '#ED7953' # pax  
                    ]
                )
        )
    )

    colors = [
        "#440154",  # <5mph
        "#21908C", # >=5mph
        "#9C179E", # delay
        "#FDE725", # freeflow
        '#ED7953' # pax  
    ]

    # colors = fig_data.color.unique().tolist
    # colors_ordered = [y for x in color_order for y in colors if y == x ]

    for r, c in zip(fig_data.full_decomp.unique(), colors):
        plot_df = fig_data[fig_data.full_decomp == r]
        fig.add_trace(
            go.Bar(x=[plot_df.route_dir, plot_df.tsp_period], y=plot_df.tot_secs_med, name=r, marker_color=c),
        )
    
    return(fig)

In [ ]:
fig1_chart_data = (
    rawnav_run_decomp_agg
    .assign(
        route_dir = lambda x: x.route + " " + x.direction_wmata_schedule
    )
    .sort_values(by = ['route','direction_wmata_schedule','tsp_period'],ascending = [True,True,False])
)

fig1 = stacked_bar(fig1_chart_data)

In [ ]:
py.offline.iplot(fig1, filename='Route Breakdown')

As an early preview of our diff-in-diff results, let's aggregate by is_tsp_route and tsp_period. 

In [ ]:
rawnav_run_slow = (
    rawnav_run_decomp_trip_2 
    # let's first summarize to trip level 
    .loc[
        (rawnav_run_decomp_trip_2.tsp_dir_time == True) & 
        (rawnav_run_decomp_trip_2.full_decomp == "<5 mph")
    ]
    .drop(['tsp_dir_time'], axis = 'columns')
    .assign(
        route_names = lambda x:
            np.where(
                x.is_tsp_route,
                "30N/30S/33",
                "31"
            )
    )
    .groupby(['route_names','tsp_period'])
    .agg(
        tot_secs = ('tot_secs','mean')
    )
    .reset_index()
    .pivot(
            index = ['tsp_period'],
            columns = 'route_names',
            values = ['tot_secs']
    )
    .reindex(['pre_tsp','post_tsp'])
)

rawnav_run_slow

We haven't sorted the above table, but this nets out to 0. 

In [ ]:
rawnav_run_slow = (
    rawnav_run_decomp_trip_2 
    # let's first summarize to trip level 
    .loc[
        (rawnav_run_decomp_trip_2.tsp_dir_time == True) & 
        (rawnav_run_decomp_trip_2.full_decomp == "Delay")
    ]
    .drop(['tsp_dir_time'], axis = 'columns')
    .assign(
        route_names = lambda x:
            np.where(
                x.is_tsp_route,
                "30N/30S/33",
                "31"
            )
    )
    .groupby(['route_names','tsp_period'])
    .agg(
        tot_secs = ('tot_secs','mean')
    )
    .reset_index()
    .pivot(
            index = ['tsp_period'],
            columns = 'route_names',
            values = ['tot_secs']
    )
    .reindex(['pre_tsp','post_tsp'])
)

rawnav_run_slow

In [ ]:
rawnav_run_motion = (
    rawnav_run_decomp_trip_2 
    # let's first summarize to trip level 
    .loc[
        (rawnav_run_decomp_trip_2.tsp_dir_time == True) & 
        (rawnav_run_decomp_trip_2.full_decomp.isin(["<5 mph",">= 5mph"]))
    ]
    .assign(
        route_names = lambda x:
            np.where(
                x.is_tsp_route,
                "30N/30S/33",
                "31"
            )
    )
    .drop(['tsp_dir_time'], axis = 'columns')
    .groupby(['route_names','tsp_period'])
    .agg(
        tot_secs = ('tot_secs','mean')
    )
    .reset_index()
    .pivot(
            index = ['tsp_period'],
            columns = 'route_names',
            values = ['tot_secs']
    ) 
    .reindex(['pre_tsp','post_tsp'])
)

rawnav_run_motion

### Stop Segment Level

What if we look at particular stop segments?

Note - we haven't looked closely at this.



In [ ]:
# THIS TRIP SEG 
rawnav_run_decomp_trip_stop = (
    rawnav_run_decomp_2 
    .groupby(
        ['filename','index_run_start',
        'trip_seg',
        'full_decomp']
    )
    .agg(
        # TODO: this is a confusing naming scheme, should revisit
        tot_secs = ('secs_tot','sum')
    )
    .reset_index()
    .merge(
        trip_details,
        on = ['filename','index_run_start'],
        how = 'left'
    )
    .groupby(['route','direction_wmata_schedule','trip_seg','tsp_period', 'full_decomp'])
    .agg(
        {'tot_secs' : [lambda x: x.quantile(.50)]}
    )
    .pipe(wr.reset_col_names)
    .rename(columns = {"tot_secs_<lambda>":"tot_secs_med"})
)

rawnav_run_decomp_trip_stop

In [ ]:
rawnav_run_decomp_trip_stop.to_csv("rawnav_run_decomp_trip_stop.csv")

## Calculating effect sizes

We'll look at the possible effects of TSP on both stop-level delay and the amount of time spent traveling at less than 5 mph. 

### Comparing Pre- and Post-TSP DID



In [ ]:
slow_speed = (
    rawnav_run_decomp_trip_2 
    .loc[
        (
            (rawnav_run_decomp_trip_2.tsp_dir_time == True) #|
            # (rawnav_run_decomp_trip_2.tsp_offdir_time == True) #we'll reintroduce this later i think
        ) &
        (rawnav_run_decomp_trip_2.full_decomp == "<5 mph")
    ]
    .assign(
        is_post_tsp = lambda x: x.tsp_period == "pre_tsp"
    )
)

slow_speed.head()

In [ ]:
smf.ols('tot_secs ~ is_tsp_route*is_post_tsp', data = slow_speed).fit().summary().tables[1] # want to add overall_dir when that's raeady

Hmm, it's looking like there is no detectable effect of TSP after it's introduced. We could try implementing a few more controls, but it's not looking promising. Many of these controls are going to be collinear with having tsp (routes and time_period with is_tsp_route), which will make any t statistic a little harder to interpret

In [ ]:
smf.ols('tot_secs ~ is_tsp_route*is_post_tsp + route + time_period', data = slow_speed).fit().summary().tables[1] # want to add overall_dir when that's raeady

What about delay ?

In [ ]:
delay_time = (
    rawnav_run_decomp_trip_2 
    .loc[
        (
            (rawnav_run_decomp_trip_2.tsp_dir_time == True) #|
            # (rawnav_run_decomp_trip_2.tsp_offdir_time == True) #we'll reintroduce this later i think
        ) &
        (rawnav_run_decomp_trip_2.full_decomp == "Delay")
    ]
    .assign(
        is_post_tsp = lambda x: x.tsp_period == "pre_tsp"
    )
)

delay_time.head()

In [ ]:
smf.ols('tot_secs ~ is_tsp_route*is_post_tsp', data = delay_time).fit().summary().tables[1] 

A little confusing because the base category is True in each case, but this says TSP makes things 23 seconds worse on average.

### Comparing CDF

Let's look at how the distribution of < 5mph travel differs between these trips

In [ ]:
rawnav_cdf_df = (
    rawnav_run_decomp_trip_2
    .loc[
        (rawnav_run_decomp_trip_2.tsp_dir_time == True) & 
        (rawnav_run_decomp_trip_2.is_tsp_route == True) &
        (rawnav_run_decomp_trip_2.full_decomp == "<5 mph")
    ]
    .drop(['tsp_dir_time'], axis = 'columns')
    .sort_values(['is_tsp_route','tot_secs'])
)
# https://stackoverflow.com/questions/65402296/python-plotly-cdf-with-frequency-distribution-data

# we'll split this into two for each of the two conditions
time_vals = rawnav_cdf_df['tot_secs'].values.tolist()
time_vals_is_tsp_route = rawnav_cdf_df.loc[rawnav_cdf_df.tsp_period == 'post_tsp', 'tot_secs'].values.tolist()
time_vals_no_tsp = rawnav_cdf_df.loc[rawnav_cdf_df.tsp_period == 'pre_tsp', 'tot_secs'].values.tolist()

hist, bin_edges_all = np.histogram(time_vals, bins = 100, density = True)

hist_is_tsp_route, bin_edges_is_tsp_route = np.histogram(time_vals_is_tsp_route, bins = 100, density = True)
hist_no_tsp, bin_edges_no_tsp = np.histogram(time_vals_no_tsp, bins = 100, density = True)

cdf_is_tsp_route = np.cumsum(hist_is_tsp_route * np.diff(bin_edges_is_tsp_route))
cdf_no_tsp = np.cumsum(hist_no_tsp * np.diff(bin_edges_no_tsp))


In [ ]:

fig3 = go.Figure(
    data=[
        go.Scatter(x=bin_edges_is_tsp_route, y=cdf_is_tsp_route, name='Post-TSP', marker_color = 'rgba(244,136,118,1)'),
        go.Scatter(x=bin_edges_no_tsp, y=cdf_no_tsp, name='Pre-TSP', marker_color = 'rgba(146,154,252,1)')
    ]
)

fig3.update_layout(
        template="simple_white",
        xaxis=dict(title_text="Travel Time (Seconds) <5 mph"),
        yaxis=dict(title_text="Percent of Trips"),
        yaxis_tickformat = '%',
        font=dict(
            size=18
        )
    )

fig3.update_yaxes(
    showgrid = True,
    gridwidth = 1,
    gridcolor = 'rgba(204,204,204,.8)'
)

fig3.update_xaxes(
    showgrid = True,
    gridwidth = 1,
    gridcolor = 'rgba(204,204,204,.8)'
)

fig3.update_yaxes(
    tickvals=[0.0,.05, .25, .50, .75, .95, 1.00]
)

py.offline.iplot(fig3, filename='CDF')

These values below should match the ones in our cross tab from before

In [ ]:
np.mean(time_vals_is_tsp_route)

In [ ]:
np.mean(time_vals_no_tsp)

These values should approximately correspond to things we see in the chart

In [ ]:
np.quantile(time_vals_is_tsp_routeroute,.95) 

In [ ]:
np.quantile(time_vals_no_tsp,.95)